# FEATURE EXTRACTIONS
- voice_position-to-predicate
- predicate-lemma_pos
- PoS of each token

In [14]:
import pandas as pd
import numpy as np

In [262]:
#header = ['Sent_ID', 'Copy_ID','ID','FORM','LEMMA','UPOS','XPOS','FEATS','HEAD','DEPREL','DEPS','MISC','UP_PRED','UP_ARGHEADS_1']
train_path = '../data_C/train_split_small.tsv'
train_df = pd.read_csv(train_path, sep='\t', header=0)
print(train_df)

     Copy_ID  Sent_ID    ID        FORM       LEMMA   UPOS  XPOS  \
0        1.0      1.0   1.0          Al          Al  PROPN   NNP   
1        1.0      1.0   2.0           -           -  PUNCT  HYPH   
2        1.0      1.0   3.0       Zaman       Zaman  PROPN   NNP   
3        1.0      1.0   4.0           :           :  PUNCT     :   
4        1.0      1.0   5.0    American    american    ADJ    JJ   
..       ...      ...   ...         ...         ...    ...   ...   
431     20.0      6.0   9.0          of          of    ADP    IN   
432     20.0      6.0  10.0          an           a    DET    DT   
433     20.0      6.0  11.0  investment  investment   NOUN    NN   
434     20.0      6.0  12.0        firm        firm   NOUN    NN   
435     20.0      6.0  13.0           .           .  PUNCT     .   

                         FEATS  HEAD    DEPREL         DEPS           MISC  \
0                  Number=Sing     0      root       0:root  SpaceAfter=No   
1                          

In [263]:
# change columns 'sent_id, Copy_ID, id' to integer
train_df = train_df.astype({'Sent_ID':'int'})
train_df = train_df.astype({'Copy_ID':'int'})
train_df = train_df.astype({'ID':'int'})

In [264]:
train_df.loc[0:5:]

,Copy_ID,Sent_ID,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,UP:PRED,UP:ARGHEADS
0,1,1,1,Al,Al,PROPN,NNP,Number=Sing,0,root,0:root,SpaceAfter=No,_,O
1,1,1,2,-,-,PUNCT,HYPH,_,1,punct,1:punct,SpaceAfter=No,_,O
2,1,1,3,Zaman,Zaman,PROPN,NNP,Number=Sing,1,flat,1:flat,_,_,O
3,1,1,4,:,:,PUNCT,:,_,1,punct,1:punct,_,_,O
4,1,1,5,American,american,ADJ,JJ,Degree=Pos,6,amod,6:amod,_,_,O
5,1,1,6,forces,force,NOUN,NNS,Number=Plur,7,nsubj,7:nsubj,_,_,ARG0


In [265]:
feature_dict_list = []

sent_id_list = train_df['Sent_ID'].unique()
print("sent_id_list", sent_id_list)
for num in sent_id_list:
    print("===== ", num, "=======")
    df_sent = train_df.loc[train_df['Sent_ID'] == num] # subset_df of df for each sent_id
    print('df_sent:',df_sent)
    print()

    copy_id_list = df_sent['Copy_ID'].unique()
    print()
    print("copy_id_list:",copy_id_list)
    for n in copy_id_list:
        df_copy = df_sent.loc[df_sent['Copy_ID'] == n] # subset_df of df_sent for each copy_id
        print()
        print("df_copy:", "Copy_ID:", n, df_copy)
        #df_copy.reset_index(drop=True, inplace=True) # reset row index in subset df

        max_wds_count = df_copy['ID'].max()
        print()
        print("max_wds_count:",max_wds_count)

        pred_id = df_copy['ID'][df_copy['UP:PRED'].str.contains('[A-Za-z0-9.]')].values[0]
        #pred_id = pred_id.tolist()

    
        print("pred_id:",pred_id, type(pred_id))

        df_copy['VOICE'] = '0' # create a new column "VOICE" and set all values to "0"

        for i in range(max_wds_count):
            feature_dict = {}

            # token
            feature_dict['token']=df_copy.iloc[i]['FORM']

           
            # extract POS, ALL TOKENS
            
            feature_dict['pos'] = df_copy.iloc[i]['XPOS']

            # extract VOICE + POSITION TO PREDICATE
            if df_copy.iloc[i]['DEPREL'] == 'nsubj:pass':
                print("if nsubj:pass: df_copy.iloc[i]['DEPREL']", pred_id-1, i)
                if i < (pred_id - 1):
                    print("if nsubj:pass i < (pred_id - 1)", "pred_id", pred_id-1, "i", i)
                    df_copy.at[i, 'VOICE'] = '1_before'
                elif i > (pred_id-1):
                    print("elif if nsubj:pass:, i > (pred_id - 1)", "pred_id", pred_id-1, "i", i)
                    df_copy.at[i, 'VOICE'] = '1_after'
                else:
                    print("else nsubj:pass =:", "pred_id", pred_id-1, 'i', i)
                    df_copy.at[i, 'VOICE'] = '1_same'
            else:
                print("else: df_copy.iloc[i]['DEPREL']", (pred_id-1), 'i', i)
                if i < (pred_id-1):
                    print("if i < (pred_id - 1)", "pred_id", pred_id-1, "i", i)
                    df_copy.at[i, 'VOICE'] = '0_before'
                elif i > (pred_id-1):
                    print("elif i > (pred_id - 1)", "pred_id", pred_id-1, "i", i)
                    df_copy.at[i, 'VOICE'] = '0_after'
                else:
                    print("else = ", "pred_id", pred_id-1, "i", i)
                    df_copy.at[i, 'VOICE'] = '0_same'
            print(">>> df_copy.iloc[i]['VOICE']",  df_copy.iloc[i]['VOICE'],"----.at ---", df_copy.at[i, 'VOICE'])
            feature_dict['voice_position-to-pred'] = df_copy.at[i, 'VOICE']


            # extract PREDICATE LEMMA + POS TAG
            if i == (pred_id - 1):
                feature_dict['pred-lemma_pos'] = f"{df_copy.iloc[i]['LEMMA']}_{df_copy.iloc[i]['XPOS']}"
            

            feature_dict_list.append(feature_dict) 
     

sent_id_list [1 2 3 4 5 6]
=====  1 =======
df_sent:     Copy_ID  Sent_ID  ID      FORM     LEMMA   UPOS  XPOS  \
0         1        1   1        Al        Al  PROPN   NNP   
1         1        1   2         -         -  PUNCT  HYPH   
2         1        1   3     Zaman     Zaman  PROPN   NNP   
3         1        1   4         :         :  PUNCT     :   
4         1        1   5  American  american    ADJ    JJ   
5         1        1   6    forces     force   NOUN   NNS   
6         1        1   7    killed      kill   VERB   VBD   
7         1        1   8    Shaikh    Shaikh  PROPN   NNP   
8         1        1   9  Abdullah  Abdullah  PROPN   NNP   
9         1        1  10        al        al  PROPN   NNP   
10        1        1  11         -         -  PUNCT  HYPH   
11        1        1  12       Ani       Ani  PROPN   NNP   
12        1        1  13         ,         ,  PUNCT     ,   
13        1        1  14       the       the    DET    DT   
14        1        1  15  preach

/var/folders/71/msbf5zws1zj3_zcm0gny0y2r0000gn/T/ipykernel_33091/2678674424.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['VOICE'] = '0' # create a new column "VOICE" and set all values to "0"
/var/folders/71/msbf5zws1zj3_zcm0gny0y2r0000gn/T/ipykernel_33091/2678674424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy.at[i, 'VOICE'] = '0_before'
/var/folders/71/msbf5zws1zj3_zcm0gny0y2r0000gn/T/ipykernel_33091/2678674424.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

     Copy_ID  Sent_ID  ID         FORM      LEMMA   UPOS XPOS  \
152        9        3   1          DPA        DPA  PROPN  NNP   
153        9        3   2            :          :  PUNCT    :   
154        9        3   3        Iraqi      iraqi    ADJ   JJ   
155        9        3   4  authorities  authority   NOUN  NNS   
156        9        3   5    announced   announce   VERB  VBD   
157        9        3   6         that       that  SCONJ   IN   
158        9        3   7         they       they   PRON  PRP   
159        9        3   8          had       have    AUX  VBD   
160        9        3   9       busted       bust   VERB  VBN   
161        9        3  10           up         up    ADP   RP   
162        9        3  11            3          3    NUM   CD   
163        9        3  12    terrorist  terrorist    ADJ   JJ   
164        9        3  13        cells       cell   NOUN  NNS   
165        9        3  14    operating    operate   VERB  VBG   
166        9        3  15

/var/folders/71/msbf5zws1zj3_zcm0gny0y2r0000gn/T/ipykernel_33091/2678674424.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['VOICE'] = '0' # create a new column "VOICE" and set all values to "0"
/var/folders/71/msbf5zws1zj3_zcm0gny0y2r0000gn/T/ipykernel_33091/2678674424.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy.at[i, 'VOICE'] = '0_before'
/var/folders/71/msbf5zws1zj3_zcm0gny0y2r0000gn/T/ipykernel_33091/2678674424.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [266]:
print(feature_dict_list)

[{'token': 'Al', 'pos': 'NNP', 'voice_position-to-pred': '0_before'}, {'token': '-', 'pos': 'HYPH', 'voice_position-to-pred': '0_before'}, {'token': 'Zaman', 'pos': 'NNP', 'voice_position-to-pred': '0_before'}, {'token': ':', 'pos': ':', 'voice_position-to-pred': '0_before'}, {'token': 'American', 'pos': 'JJ', 'voice_position-to-pred': '0_before'}, {'token': 'forces', 'pos': 'NNS', 'voice_position-to-pred': '0_before'}, {'token': 'killed', 'pos': 'VBD', 'voice_position-to-pred': '0_same', 'pred-lemma_pos': 'kill_VBD'}, {'token': 'Shaikh', 'pos': 'NNP', 'voice_position-to-pred': '0_after'}, {'token': 'Abdullah', 'pos': 'NNP', 'voice_position-to-pred': '0_after'}, {'token': 'al', 'pos': 'NNP', 'voice_position-to-pred': '0_after'}, {'token': '-', 'pos': 'HYPH', 'voice_position-to-pred': '0_after'}, {'token': 'Ani', 'pos': 'NNP', 'voice_position-to-pred': '0_after'}, {'token': ',', 'pos': ',', 'voice_position-to-pred': '0_after'}, {'token': 'the', 'pos': 'DT', 'voice_position-to-pred': '0_